In [ ]:

import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import geopandas as gpd
import cupy as np
import cupyx.scipy as scipy
import h3pandas
import random

# from dask.distributed import Client

# cluster = LocalCUDACluster()
# client = Client(cluster)
import sklearn.cluster

In [ ]:
nypd_data = pd.read_csv('../../data/clean/police_data/NYPDcleanupfinal.csv')

In [ ]:
# def plot_elbow(kmean, X):
#     centroids = [k.cluster_centers_ for k in kmean]
#     D_k = [cdist(X, center, 'euclidean') for center in centroids]
#     dist = [np.min(D, axis=1) for D in D_k]
#
#     # Total with-in sum of square
#     wcss = [sum(d ** 2) for d in dist]
#     tss = sum(pdist(X) ** 2) / X.shape[0]
#     bss = tss - wcss
#
#     plt.subplots(nrows=1, ncols=1, figsize=(8, 8))
#     ax = plt.subplot(1, 1, 1)
#     ax.plot(Ks, bss / tss * 100, 'b*-')
#     plt.grid(True)
#     plt.xlabel('Number of clusters')
#     plt.ylabel('Percentage of variance explained (%)')
#     plt.title('Elbow for KMeans clustering')
#     plt.show()

In [ ]:
sales_data = pd.read_csv('../../data/clean/SalesFinal.csv')

In [ ]:
building_data = gpd.read_file('../../data/clean/building_stats.csv')
park_mask = gpd.read_file('../../data/clean/park_hexes.csv')
building_data = building_data.set_index('hex_id')
park_mask = park_mask.set_index('hex_id')

In [ ]:
hex_files = ('bronx_hex.csv', 'brooklyn_hex.csv', 'manhattan_hex.csv', 'queens_hex.csv', 'staten_hex.csv')
hex_dfs = []
for f in hex_files:
    hex_dfs = hex_dfs + list(pd.read_csv(f"../../data/clean/h3_index/{f}")[f.split('.')[0]])


In [ ]:
# hex_dfs

In [ ]:
nypd_data = nypd_data[nypd_data['h3geo'].isin(hex_dfs)]
nypd_data['RPT_DT'] = pd.to_datetime(nypd_data['RPT_DT'])
# nypd_data = nypd_data[nypd_data['RPT_DT'].dt.year == 2021]

In [ ]:
sales_data = sales_data[sales_data['h3geo'].isin(hex_dfs)]
sales_data['BUILDING_CLASS'] = sales_data['BUILDING CLASS AT TIME OF SALE']

selected_cols_sales = sales_data[[
    'h3geo',
    'BUILDING_CLASS',
    'YEAR BUILT',
    'SALE PRICE'
    ]]

In [ ]:
clean_nypd = nypd_data.set_index('h3geo')
clean_nypd['lat'] = clean_nypd['Latitude']
clean_nypd['lon'] = clean_nypd['Longitude']
clean_nypd = clean_nypd.drop(['Latitude', 'Longitude'], axis=1)

In [ ]:
selected_cols_nypd = clean_nypd[[
    'CMPLNT_FR_TM',
    'RPT_DT',
    'KY_CD',
    'OFNS_DESC',
    'CRM_ATPT_CPTD_CD',
    'LAW_CAT_CD',
    'JURIS_DESC',
    ]]
selected_cols_nypd = selected_cols_nypd[selected_cols_nypd['OFNS_DESC'].isnull() == False]


In [ ]:
def get_hour_of_day(thetime):
    strips = thetime.split(':')
    hour = int(strips[0])
    if hour < 5:
        return 'NIGHT'
    elif hour < 11:
        return 'MORNING'
    elif hour < 17:
        return 'DAY'
    elif hour < 23:
        return 'EVENING'
    else:
        return 'NIGHT'


In [ ]:
# selected_cols_nypd['FUZZY_TIME'] = selected_cols_nypd['CMPLNT_FR_TM'].apply(get_hour_of_day)
selected_cols_nypd = selected_cols_nypd.drop(['CMPLNT_FR_TM', 'RPT_DT'], axis=1)

In [ ]:
nypd_fe = pd.get_dummies(selected_cols_nypd,
                         # columns=['OFNS_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'KY_CD', 'FUZZY_TIME'],
                         columns=['OFNS_DESC', 'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'KY_CD'],
                         prefix_sep='_')


In [ ]:
nypd_fe = nypd_fe.groupby('h3geo').sum()

In [ ]:
selected_cols_sales['BUILDING_CLASS'] = selected_cols_sales['BUILDING_CLASS'].apply(lambda x: x[0])

In [ ]:
sales_fe = pd.get_dummies(selected_cols_sales, columns=['BUILDING_CLASS']).set_index('h3geo')
sales_fe['COUNT'] = 1
sales_fe2 = sales_fe.groupby('h3geo').sum()
sales_fe = sales_fe.groupby('h3geo').sum()
sales_fe['MEAN_BLD_AGE'] = 2023 - (sales_fe['YEAR BUILT'] / sales_fe['COUNT'])
sales_fe['MEAN_SALE_PRICE'] = sales_fe['SALE PRICE'] / sales_fe['COUNT']

In [ ]:
sales_fe3 = sales_fe2.div(sales_fe2['COUNT'], axis=0)
sales_fe2 = sales_fe2.drop(['YEAR BUILT', 'SALE PRICE', 'COUNT'], axis=1)

# sales_fe2

In [ ]:
sales_fe = sales_fe.drop(['YEAR BUILT', 'SALE PRICE'], axis=1)

In [ ]:
sales_fe = sales_fe[['COUNT', 'MEAN_BLD_AGE', 'MEAN_SALE_PRICE']]
sales_fe = sales_fe.join(sales_fe2)

In [ ]:
helper_outer = sales_fe.join(nypd_fe, how='outer').join(building_data, how='outer')
helper_inner = sales_fe.join(nypd_fe, how='inner')
helper_inner = nypd_fe

wopt = pd.DataFrame(hex_dfs, columns=['hex_id'])
wopt = wopt.set_index('hex_id')
all_hexes = wopt.join(helper_outer, how='inner')
all_hexes.shape
wopt

In [ ]:
helper_all = pd.DataFrame(hex_dfs, columns=['h3geo']).set_index('h3geo')
police_all = helper_all.join(nypd_fe, how='inner')

In [ ]:
helper = pd.read_csv('../../data/clean/h3_index/brooklyn_hex.csv')
helper.columns = ['h3geo']
helper = helper.set_index('h3geo')
# helper = helper.join(sales_fe, how='left')

In [ ]:
# helper = helper.join(nypd_fe, how='left')

In [ ]:
# helper_outer = helper_outer_nonan =

In [ ]:
import folium
from libpysal.weights import Queen, KNN
from esda.moran import Moran

from sklearn.cluster import KMeans

In [ ]:
cluster_variables = ['BldgArea', 'YearBuilt', 'AssessTot', 'ResArea'

                     ]

In [ ]:
# #1.1 data preparation
# X = helper_inner.iloc[:, 1:].values
# # Using the elbow method to find the optimal number of clusters
# wcss = []
# for i in range(1, 60):
#     kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
#     kmeans.fit(X)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 60), wcss)
# plt.title('The Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')
# plt.show()

In [ ]:
df_map = helper_all.join(helper_inner, how='left').fillna(method='ffill')
# df_map = helper.join(helper_inner, how='inner')
# df_map = helper_inner
w = Queen.from_dataframe(df_map.h3.h3_to_geo_boundary().reset_index())

clean = df_map.h3.h3_to_geo_boundary().reset_index().drop(w.islands)
w2 = Queen.from_dataframe(clean)
w = w2
w2

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import robust_scale

# db_scaled = robust_scale(clean.set_index('h3geo').drop(['COUNT', 'geometry'], axis=1))
db_scaled = robust_scale(clean.set_index('h3geo').drop(['geometry'], axis=1))

# # Set the seed for reproducibility
# np.random.seed(123456)
# # Specify cluster model with spatial constraint
# model = AgglomerativeClustering(
#     linkage='average', connectivity=w.sparse, n_clusters=5, compute_full_tree=True
#     )
# # Fit algorithm to the data
# model.fit(db_scaled)

In [ ]:
clean['ward5wq'] = model.labels_
# Setup figure and ax
f, ax = plt.subplots(1, figsize=(30, 30))
# Plot unique values choropleth including a legend and with no boundary lines
clean.plot(column='ward5wq', legend=True, linewidth=0, ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
clean[, 2:]

In [ ]:
# import plotly.express as px
# import plotly.graph_objects as go

plt.rcParams['figure.figsize'] = (12, 8)
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn.preprocessing import StandardScaler

In [ ]:
db_scaled = robust_scale(all_hexes)
all_hexes

In [ ]:
db = DBSCAN(eps=0.5, min_samples=6)

In [ ]:
db_p = db.fit_predict(db_scaled)
helper

In [ ]:
dew_it = pd.DataFrame(db_p).set_index(police_all.index)
dew_it

In [ ]:
import hdbscan

In [ ]:
db_scaled_copy = db_scaled.copy()
db_scaled_copy = pd.DataFrame(db_scaled_copy)
db_scaled_copy = db_scaled_copy.set_index(police_all.index)
db_scaled_copy.columns = police_all.columns
db_scaled_copy['Cluster'] = db_p
# final = db_scaled_copy.groupby('Cluster').mean()
# # final = final.transpose().reset_index()

In [ ]:
db_scaled_copy

In [ ]:
db_scaled_copy = db_scaled_copy.h3.h3_to_geo_boundary()
db_scaled_copy.index.names = ['hex_id']

db_scaled_copy_2 = mask(db_scaled_copy.reset_index(), the_mask)
# Setup figure and ax
f, ax = plt.subplots(1, figsize=(30, 30))
# Plot unique values choropleth including a legend and with no boundary lines
db_scaled_copy_2.plot(column='Cluster', categorical=True, legend=True, linewidth=0, ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [ ]:
from sklearn.decomposition import PCA

_CRS = 'espg:4326'

In [ ]:
# subway_map = pd.read_csv('../../data/raw/subway_lines.csv')
# subway_map['geometry'] = subway_map['the_geom']
# subway_map = gpd.GeoDataFrame(subway_map,crs=_CRS,geometry='the_geom')
# f, ax = plt.subplots(1, figsize=(10, 40))
# # Plot unique values choropleth including a legend and with no boundary lines
# subway_map.plot(column='NAME', categorical=True, legend=False, linewidth=0.5, ax=ax)
# # Remove axis
# ax.set_axis_off();
# Display the map




In [ ]:
PCA(db_scaled).score()
db_scaled.shape

In [ ]:
neighborhood_centroids = pd.read_csv('../../data/clean/neighborhood_centroids.csv')
from h3 import h3_distance
import h3
from IPython.display import clear_output
from sklearn.metrics.pairwise import euclidean_distances, nan_euclidean_distances
import random, os
from datetime import datetime


class CellFactory:

    def __init__(self, hex_list, data_list, neighborhood_id_list, output_dir_name):
        """ Initializes new object
        :param hex_list:
        :param data_list:
        :param neighborhood_id_list:
        """
        now = datetime.now()
        self.output_dir_name = f"{output_dir_name}_{now.strftime('%D-%H-%M-%S')}"
        self.results_df_list = []

        self.hex_list = hex_list
        self.data_list = data_list
        self.neighborhood_id_list = neighborhood_id_list.set_index('hex_id')

        self.neighborhood_id_list = self.neighborhood_id_list.join(self.hex_list, how='left')
        # assert sum([~self.neighborhood_id_list['hex_id'].isin(self.hex_list)]) == 0

        self.cell_dict = {}
        self.neighborhood_dict = {}

    def build_cell_dict(self, all_cells=False):
        if all_cells:
            index_list = self.hex_list.join(self.data_list, how="left", lsuffix='L')
        else:
            index_list = self.hex_list.join(self.data_list, how="inner", lsuffix='L')

        # index_list = self.hex_list.join(self.data_list, how="inner", lsuffix='L')
        # self.neighborhood_id_list = self.neighborhood_id_list.filter(items=index_list['hex_id'], axis=0).filter(i)

        for index, row in index_list.iterrows():
            hex_id = row['hex_id']
            #TODO: DON"T HARD CODE THIS, ITS BAD.  Set the correct pre and post conditions for the data dfs!!
            # TODO: STILL GOTTA FIX THIS, DOOFUS
            self.cell_dict[hex_id] = HexCell(hex_id, self, data_values=list(row[2:]))

        self.trim_neighbors()

    def generate_neighborhoods(self):
        self.neighborhood_dict = {}
        for hood_id, hood_name in self.neighborhood_id_list.iterrows():
            self.neighborhood_dict[hood_id] = GreedyNeighborhood(self.cell_dict[hood_id], hood_name[0], factory=self)

    def get_cell_dict(self):
        return self.cell_dict

    def trim_neighbors(self):
        """ Void method that trims cell neighbors
        \n :Pre Cell Dict has been populated with list of all viable cells, and each cell has had their neighbor lists populated.
        \n :Post each cell in self.cell_dict only has neighbors who are also in the cell dict.
        """
        for key in self.cell_dict.keys():
            self.cell_dict[key].neighbor_ids = [the_id for the_id in self.cell_dict[key].neighbor_ids if
                                                the_id in self.cell_dict.keys()]
            self.cell_dict[key].wanted_cells = [the_id for the_id in self.cell_dict[key].neighbor_ids if
                                                the_id in self.cell_dict.keys()]

    def simulate(self, n=2, graph_during=False):
        # For the number of rounds specified by the user:
        for i in range(1, n + 1):
            # For each neighborhood in the list
            for neighborhood_id in self.neighborhood_dict.keys():
                neighborhood = self.neighborhood_dict[neighborhood_id]
                print(f"Simulating round {i} of {n}")
                print(f"Current Neighborhood: {neighborhood.get_name()}")
                if neighborhood.cell_count:
                    neighborhood.play_turn()
                clear_output(wait=True)
                self.neighborhood_dict[neighborhood_id] = neighborhood.update_factory()

            self.results_df_list = self.results_df_list + [self.get_results()]
            if graph_during:
                self.plot_neighborhoods(i)

    def get_results(self):
        results_list = []
        for hood in self.neighborhood_list:
            # print(hood.output_cell_map())
            results_list = results_list + [hood.output_cell_map()]

        output = pd.concat(results_list)
        return output

    def plot_neighborhoods(self, iteration):
        results = self.get_results()
        # results = results[~results['hex_id'].isin(list(park_mask.reset_index()['hex_id']))]
        results = results.set_index('hex_id')
        results_plot = df.h3.h3_to_geo_boundary()

        # Setup figure and ax
        f, ax = plt.subplots(1, figsize=(10, 40))
        # Plot unique values choropleth including a legend and with no boundary lines
        results_plot.plot(column='name', categorical=True, legend=False, linewidth=0.5, ax=ax)
        # Remove axis
        ax.set_axis_off()
        # Display the map
        plt.show()
        fstring = f'../../out/images/plt/{self.output_dir_name}'
        os.mkdir(fstring)
        plt.savefig(f"{fstring}--{iteration}")

    def do_it_myself(self):
        df = pd.DataFrame()
        for key, cell in self.cell_dict.items():
            df[key] = cell.neighborhood_id

        return df

    # determine which
    # this would be the point to inject the annealing step, making subomptimal decisions probabilistically.
    def arbitrate_cell_flip(self, requested_hex, new_neighborhood_id, new_affinity, temperature=1):
        cell = self.cell_dict[requested_hex]
        assert cell.neighborhood_id != new_neighborhood_id

        # if the cell is unowned, allow the flip
        if cell.neighborhood_id == -1:
            cell.neighborhood_id = new_neighborhood_id
            self.cell_dict[requested_hex] = cell
        else:
            current_affinity = cell.affinity
            rand = random.uniform(0, 1)

            if new_affinity > current_affinity:
                if rand < temperature:
                    cell_dict[requested_hex].neighborhood_id = new_neighborhood_id
                    self.neighborhood_dict[cell.neighborhood_id] = self.neighborhood_dict[
                        cell.neighborhood_id].update_edges_after_losing_cell(requested_hex)
            else:
                if rand > temperature:
                    cell_dict[requested_hex].neighborhood_id = new_neighborhood_id
                    self.neighborhood_dict[cell.neighborhood_id] = self.neighborhood_dict[
                        cell.neighborhood_id].update_edges_after_losing_cell(requested_hex)

        current_neighborhood_id = cell.neighborhood_id


class CellFactoryRandomStart(CellFactory):

    def __init__(self, hex_list, data_list, number_of_clusters, ):
        tuple_list = self.load_hexes(hex_list)
        for a_tuple in tuple_list:
            hexes, proportion = a_tuple
            sample_size = round(proportion * number_of_clusters)
            selected = random.sample(hexes, sample_size)

    def load_hexes(self, hex_list):
        hex_files = ('bronx_hex.csv', 'brooklyn_hex.csv', 'manhattan_hex.csv', 'queens_hex.csv', 'staten_hex.csv')
        hex_tuples = []
        sum_hexes = len(hex_list)
        for f_name in hex_files:
            boro_hex_list = list(pd.read_csv(f"../../data/clean/h3_index/{f_name}")[f_name.split('.')[0]])
            boro_hex_list = boro_hex_list[boro_hex_list in hex_list]
            boro_size = len(boro_hex_list)
            hex_tuples = hex_tuples + [tuple(boro_hex_list, boro_size / sum_hexes)]

        return hex_tuples


CellFactoryRandomStart(in_hex, db_scaled, 5)

In [ ]:
class HexCell:
    """

    """

    def __init__(self, hex_id, factory, data_values=([None] * cell_len)):
        self.hex_id = hex_id
        self.neighbor_ids = list(h3.k_ring(hex_id))
        self.data_values = data_values
        self.affinity = -1
        self.neighborhood_id = -1
        self.oob = True
        self.factory = factory

        # border score for surface tension metrics
        self.border_score = 0

    def set_data_values(self, data_values, start_index=0, end_index=cell_len):
        self.data_values = data_values

    def get_data_values(self, fill_nan=True, zeroes=False):
        """
        Getter method for the data values of the cell.  Can also be used to
        :param fill_nan:
        :param zeroes:
        :return:
        """
        if fill_nan:
            newvals = []
            for val in self.data_values:
                if pd.isna(val):
                    newvals = newvals + [random.uniform(0, 1)]
                else:
                    newvals = newvals + [val]
            return newvals
        else:
            return self.data_values

    def detect_border(self):
        for id in self.neighbor_ids:
            if self.factory.cell_dict[id].neighborhood_id != self.neighborhood_id:
                return True

        # for id in self.neighbor_ids:
        #           if id in self.factory.cell_dict.keys():
        #               if self.factory.cell_dict[id].neighborhood_id != self.neighborhood_id:
        #                   return True

        return False





In [ ]:
from copy import copy


class GreedyNeighborhood:
    """

    """

    def __init__(self, starting_cell, name, factory, cell_count=1):

        #
        self.centroid_cell = copy(starting_cell)
        self.centroid_id = self.centroid_cell.hex_id
        self.centroid_cell.neighborhood_id = self.centroid_id

        # store neighborhood name
        self.name = name

        # initialize neighborhood mean as the value of the initial cell.
        self.neighborhood_mean = (self.centroid_cell.get_data_values(True))
        self.factory = factory
        self.factory.cell_dict[self.centroid_id] = self.centroid_cell

        ## initializing lists of cells that are part of the neighborhood
        self.border_cells = [self.centroid_id]
        self.interior_cells = [self.centroid_id]
        self.wanted_cells = []

        self.cell_count = cell_count
        self.neighborhood_affinity = 1

        # print (self.centroid_cell.neighbor_ids)

    def play_turn(self):
        self.neighborhood_mean = self.recompute_mean()
        self.claim_cells()
        if len(self.wanted_cells):
            self.claim_cells()

    def resolve_ownership_disputes(self):
        self.add_to_interior([cell for cell in self.get_owned_cells() if
                              self.factory.cell_dict[cell].neighborhood_id == self.centroid_id])

    # gets a list of all neighbors of bordering cells  and then trims any that are already in the neighborhood be
    def update_edges_after_gaining_cell(self, expanded_hex_id):
        self.cell_count += 1
        self.wanted_cells.remove(expanded_hex_id)
        self.border_cells = self.border_cells + [expanded_hex_id]

        expanded_cell = self.factory.cell_dict(expanded_hex_id)

        # counter for surface tension
        border_rating = 0

        # for each of the new cell's neighbors
        for nb_id in expanded_cell.neighbor_ids:
            nb_cell = self.factory.cell_dict[nb_id]
            # check if the neighbor is in the current
            if nb_cell.neighborhood_id != self.centroid_id:
                # if it's not in the current neighborhood and not already in the wanted cells list, add it to the wanted cells list
                if nb_id not in self.wanted_cells:
                    self.wanted_cells = self.wanted_cells + [nb_id]
            else:  # it must be a neighbor, and therefore it must be in the border cells list
                border_rating += 1  # increment border rating

                # check if all of a border cell's neighbors are in this same neighborhood
                has_gaps = pd.Series([nb_cell.neighbor_ids in self.get_owned_cells()]).all()
                if ~has_gaps:  # if they are, move it from the border to the body
                    self.border_cells = self.border_cells.remove(nb_id)
                    self.interior_cells = self.interior_cells + [nb_id]
        return self

    def update_edges_after_losing_cell(self, lost_hex_id):
        self.cell_count -= 1

        # update the lists with the lost new cell id
        self.wanted_cells = self.wanted_cells + [lost_hex_id]
        self.border_cells = self.border_cells.remove(lost_hex_id)

        # get lost cell from cell dict
        lost_cell = self.factory.cell_dict(lost_hex_id)

        # counter for surface tension
        border_rating = 0

        # # TODO: MAKE THIS CODE WAY MORE PYTHONIC?  LOTS OF FOR LOOPS AND LISTS, COULD PROBABLY SIMPLIFY WITH LIST COMPREHENSION
        # for each of the lost cell's neighbors
        for nb_id in lost_cell.neighbor_ids:
            nb_cell = self.factory.cell_dict[nb_id]
            # check if the neighbor is in the current
            if nb_cell.neighborhood_id != self.centroid_id:
                # if it's not in the current neighborhood and not already in the wanted cells list, add it to the wanted cells list
                if nb_id not in self.wanted_cells:
                    self.wanted_cells = self.wanted_cells + [nb_id]

            else:  # it must be a neighbor, and therefore it must be in the border cells list
                border_rating += 1  # increment border rating
                # check if all of a border cell's neighbors are in this same neighborhood
                has_gaps = pd.Series([nb_cell.neighbor_ids in self.get_owned_cells()]).all()

                if ~has_gaps:  # if they are, move it from the interior list to the border list
                    self.border_cells = self.border_cells + [nb_id]
                    self.interior_cells = self.interior_cells.remove(nb_id)

        return self

    # def get_wanted_cells(self):
    #     for cell_id in self.border_cells:
    #         cell = self.factory.cell_dict[cell_id]
    #         potentials = cell.neighbor_ids
    #         potentials = np.setdiff1d(potentials, self.get_owned_cells())
    #         self.wanted_cells = list(self.wanted_cells) + list(potentials)
    #         self.wanted_cells = list(set(self.wanted_cells))

    def get_owned_cells(self):
        return self.border_cells + self.interior_cells

    def is_connected(self):
        pass

    def get_distance(self, X, Y, Y_norm_sqd):
        return euclidean_distances(X, Y, Y_norm_squared=Y_norm_sqd, squared=True)

    def get_nan_distance(self, X, Y):
        return nan_euclidean_distances(X, Y, squared=True)

    def claim_cells(self):
        assert len(self.wanted_cells) > 0

        Y = np.array(self.neighborhood_mean).reshape(1, -1)
        X = np.array([self.factory.cell_dict[cell].get_data_values(True) for cell in self.wanted_cells]).reshape(
            len(self.wanted_cells), -1)

        rez = pd.DataFrame(nan_euclidean_distances(X, Y, squared=True), columns=['dist'])

        rez['current_affinity'] = [self.factory.cell_dict[cell].affinity for cell in self.wanted_cells]
        rez['delta_affinity'] = rez['dist'] - rez['current_affinity']
        rez['id'] = self.wanted_cells

        wantMax = True
        if wantMax:
            # temp = (rez[rez['delta_affinity'] == rez['delta_affinity'].max()])
            temp = (rez[rez['dist'] == rez['dist'].max()])
        else:
            temp = rez[(rez['delta_affinity'] > 0)]

        claiming_cells = temp['id']
        for _, row in temp.iterrows():
            cell_flip_occurred = self.factory.arbitrate_cell_flip(row['id'], self.centroid_id, row['dist'])
            if cell_flip_occurred:
                self.update_edges_after_gaining_cell(row['id'])
            if wantMax:
                break

            # self.factory.cell_dict[row['id']].affinity = row['dist']
            # self.factory.cell_dict[row['id']].neighborhood_id = self.centroid_id

        """
        TODO FIX THIS GARBANZO BEANS
        """

        # self.wanted_cells = np.setdiff1d(self.wanted_cells, claiming_cells)

        # self.determine_border()
        # self.add_to_interior(claiming_cells)

    def determine_border(self):
        new_border = []
        for cell_id in self.border_cells:
            if self.factory.cell_dict[cell_id].neighborhood_id == self.centroid_id:
                new_border = new_border + [cell_id]

    def add_to_interior(self, new_cell_ids):
        check_list = []
        for cell_id in new_cell_ids:
            check_list = check_list + self.factory.cell_dict[cell_id].neighbor_ids

        # check_list  = [x for x in check_list if x in self.factory.cell_dict.keys()]

        to_interior_list = []
        for cell_id in check_list:
            cell = self.factory.cell_dict[cell_id]
            if cell.neighborhood_id == self.centroid_id:
                if not cell.detect_border():
                    to_interior_list = to_interior_list + [cell_id]
                else:
                    self.border_cells = self.border_cells + [cell_id]

        self.interior_cells = list(set(self.interior_cells + to_interior_list))
        self.border_cells = list(set(self.border_cells))

        # TODO: HANDLE BORDER INCURSIONS

    # def a_b_test(self, hex_id):
    #
    #     self.neighborhood_mean = self.recompute_mean()
    #     Y = np.array(self.neighborhood_mean).reshape(1, -1)
    #     X = np.array([self.factory.cell_dict[cell].get_data_values(True) for cell in self.wanted_cells]).reshape(
    #         len(self.wanted_cells), -1)
    #
    #     rez = pd.DataFrame(nan_euclidean_distances(X, Y, squared=True), columns=['dist'])

    def recompute_mean(self, fill_nan=False, compare=False):
        """
        Recomputes the mean every time for the whole neighborhood.  This sucks, and is slow.  Better to keep a running tab of the mean, and then multiply by number of constituent cells to get the sum, and add/subtract from that sum.  Much faster, but I'm being lazy because I'm...well..lazy?
        :param nan:
        :param compare: False, or h3 hex index.
        :return:
        """
        if compare:
            assert h3.h3_is_valid(compare)
        owned_cells = self.get_owned_cells()
        weighted_sum_list = None

        # compare_flag = False
        # compare_weighted_sum_list = None

        for cell_id in owned_cells:
            cell = self.factory.cell_dict[cell_id]
            weight = 1 / (2 ** h3_distance(self.centroid_id, cell.hex_id))
            component_list = tuple([x * weight for x in cell.get_data_values(True)])

            # print(component_list)
            # if compare ==  cell_id:
            #     compare_flag = True
            #     compare_weighted_sum = weighted_sum_list
            # elif compare_flag:
            #     compare_weighted_sum_list = self.pairwise_tuple_combine(compare_weighted_sum_list, component_list)

            weighted_sum_list = self.pairwise_tuple_combine(weighted_sum_list, component_list)

        # if compare:
        #     counter_add = -1
        #     if ~compare_flag:
        #         counter_add = 1
        #         cell = self.factory.cell_dict[cell_id]
        #         weight = 1 / (2 ** h3_distance(self.centroid_id, cell.hex_id))
        #         component_list = tuple([x * weight for x in cell.get_data_values(True)])
        #         compare_weighted_sum_list = self.pairwise_tuple_combine(compare_weighted_sum_list, component_list)
        #
        #     mean = [x / self.cell_count for x in weighted_sum_list]
        #     compared_mean = [x / (self.cell_count + counter_add) for x in compare_weighted_sum_list]
        #     self.neighborhood_mean = mean
        #     return compared_mean
        # else:
        #     mean = [x / self.cell_count for x in weighted_sum_list]
        #     return mean
        div_list = [x / self.cell_count for x in weighted_sum_list]

        return self.neighborhood_mean

    def get_name(self):
        return self.name

    def update_factory(self):
        return self

    def output_cell_map(self):
        out = pd.DataFrame(self.get_owned_cells(), columns=['hex_id'])
        out['name'] = self.name
        out['centroid_id'] = self.centroid_id
        return out

    def output_data(self):
        self.recompute_mean()
        return [self.centroid_id, self.get_owned_cells(), self.neighborhood_mean]

    @staticmethod
    def pairwise_tuple_combine(tup_a, tup_b):
        if tup_a == None:
            # print("Warning: tup_a is 'None'")
            return tup_b
        out = tuple([i + j for i, j in zip(tup_a, tup_b)])
        return out

    def surface_tension_check(self, konstant):
        """
        Surface Tension = Surface Force/ edge length of force
        :param konstant:
        :return:
        """
        pass


In [ ]:
db_scaled.describe()

In [ ]:
hood_df = pd.read_csv('../../data/clean/neighborhood_centroids.csv')

hood_df['hex_id'] = hood_df['h3_10']
hood_df = pd.DataFrame(hood_df[['hex_id', 'Name']])

# hex_list = pd.read_csv('../../data/clean/h3_index/manhattan_hex.csv')
# hex_list['hex_id'] = hex_list['manhattan_hex']
# hex_list = pd.DataFrame(hex_list['hex_id'])
# data_list = helper_inner.join(hex_list.set_index('hex_id'), how='inner')

hex_list = wopt.reset_index()

workpls = list(wopt.reset_index()['hex_id']) + list(hood_df['hex_id'])[1:]
# hood_df = [print(row['hex_id']) for _, row in hood_df.iterrows()]
# hood_df = [([row['hex_id'], row['Name']]) for _, row in hood_df.iterrows() if row['hex_id'].isin(list(hex_list['hex_id']))]
workpls = list(set(workpls))
workpls = workpls
hex_list = pd.DataFrame(workpls, columns=['hex_id'])

In [ ]:
# db_scaled = pd.DataFrame(robust_scale(helper_inner, axis=1)).set_index(helper_inner.index)
# hood_df2 = hood_df.filter(items=wopt.reset_index())
the_filter = pd.read_csv('../../data/clean/h3_index/manhattan_hex.csv', names=['hex_id'], header=0)
the_mask = pd.DataFrame(park_mask.index, columns=['hex_id'])


def filter(df, filter):
    return df[df['hex_id'].isin(filter['hex_id'])]


def mask(df, mask):
    return df[~df['hex_id'].isin(mask['hex_id'])]


in_hex = pd.DataFrame(workpls, columns=['hex_id'])
in_hex = mask(filter(in_hex, the_filter), the_mask)
in_hex = in_hex.reset_index(drop=True)

hood_df = mask(filter(hood_df, the_filter), the_mask)

db_scaled = pd.DataFrame(robust_scale(all_hexes)).set_index(all_hexes.index)
db_scaled.index.names = ['hex_id']
db_scaled = filter(db_scaled.reset_index(), the_filter)
db_scaled = mask(db_scaled, the_mask)
db_scaled = db_scaled.set_index('hex_id')


## *I'm fixing a hole...*

In [ ]:
def fixing_a_hole():
    pass

In [ ]:
pca3 = PCA(n_components=3)
pcps = pca3.fit_transform(db_scaled)
# principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'PC3','PC4','PC5'])
principalDf = pd.DataFrame(data=pcps, columns=['principal component 1', 'principal component 2', 'PC3'])
principalDf.head()

In [ ]:
# in_hex.reset_index(drop=True)

In [ ]:
pd.DataFrame(workpls, columns=['hex_id'])['hex_id'].str.len().min()

In [ ]:
db_scaled.index.names = ['hex_id']

In [ ]:
the_factory = CellFactory(in_hex, db_scaled, hood_df, "test_run_1")
the_factory.build_cell_dict(all_cells=True)
the_factory.get_cell_dict()
the_factory.generate_neighborhoods()


In [ ]:
hood_df

In [ ]:
the_factory.simulate(10)

In [ ]:
# hood_df.filter(items=hex_df).h3.h3_toe4_geo_boundary().plot()

In [ ]:
the_factory.get_results()
len(the_factory.cell_dict.keys())

In [ ]:
df = the_factory.get_results()
df = df[~df['hex_id'].isin(list(park_mask.reset_index()['hex_id']))]

df = df.set_index('hex_id')

In [ ]:
df_plot = df.h3.h3_to_geo_boundary()

# Setup figure and ax
f, ax = plt.subplots(1, figsize=(10, 40))
# Plot unique values choropleth including a legend and with no boundary lines
df_plot.plot(column='name', categorical=True, legend=False, linewidth=0.5, ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

In [75]:
the_factory.get_results()
len(the_factory.cell_dict.keys())

AttributeError: 'CellFactory' object has no attribute 'neighborhood_list'

In [76]:
df = the_factory.get_results()
df = df[~df['hex_id'].isin(list(park_mask.reset_index()['hex_id']))]

df = df.set_index('hex_id')

AttributeError: 'CellFactory' object has no attribute 'neighborhood_list'

In [77]:
df_plot = df.h3.h3_to_geo_boundary()

# Setup figure and ax
f, ax = plt.subplots(1, figsize=(10, 40))
# Plot unique values choropleth including a legend and with no boundary lines
df_plot.plot(column='name', categorical=True, legend=False, linewidth=0.5, ax=ax)
# Remove axis
ax.set_axis_off()
# Display the map
plt.show()

NameError: name 'df' is not defined